<a href="https://colab.research.google.com/github/JieShenAI/torch/blob/main/huggingface/example/translation/%E8%8B%B1%E6%B1%89%E4%BA%92%E8%AF%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* huggingface翻译 https://huggingface.co/learn/nlp-course/zh-CN/chapter7/4

In [ ]:
!pip install transformers==4.24.0
!pip install SentencePiece
!pip install sacremoses
!pip install datasets
!pip install evaluate
!pip install sacrebleu

In [ ]:
!pip install huggingface_hub==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.15.1
    Uninstalling huggingface-hub-0.15.1:
      Successfully uninstalled huggingface-hub-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.13.1 requires huggingface-hub<1.0.0,>=0.11.0, but you have huggingface-hub 0.10.1 which is incompatible.


In [ ]:
import torch
from torch import nn
from transformers import (
    MarianTokenizer,
    MarianMTModel,
    MarianConfig,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    T5ForConditionalGeneration,
    )

In [ ]:
# MarianMTModel?

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_model(src, trg):
    # model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
    model_name = "jieshenai/zh_en_translation"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    return model, tokenizer

## 模型英文->中文
def get_model_en_zh():
    src = "en"  # source language
    trg = "zh"  # target language
    return get_model(src, trg)


## 模型中文->英文
def get_model_zh_en():
    src = "zh"  # source language
    trg = "en"  # target language
    return get_model(src, trg)


In [ ]:
## 翻译
def trans(model, tokenizer, sample_text):
    batch = tokenizer([sample_text], return_tensors="pt")
    for k,v in batch.items():
      batch[k] = v.to(device)
    generated_ids = model.generate(**batch)
    text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return text

In [ ]:
model, tokenizer = get_model_zh_en()
# model.init_weights()

In [ ]:
model

In [ ]:
isinstance(model.state_dict(), dict)

True

In [ ]:
isinstance(model, nn.Module)

True

In [ ]:
model.state_dict()["final_logits_bias"]

tensor([[ 6.3965, -0.8680, -2.8505,  ..., -2.2678, -2.2489,  0.0000]])

In [ ]:
s = "model.encoder.embed_tokens.weight"
model.state_dict()[s]

tensor([[-0.0175,  0.0060, -0.0120,  ..., -0.0074, -0.0449, -0.0565],
        [ 0.0209,  0.0023, -0.0043,  ...,  0.0590, -0.0862, -0.0706],
        [-0.0041,  0.0053,  0.0189,  ...,  0.0026, -0.0172, -0.0435],
        ...,
        [ 0.0054,  0.0304,  0.0070,  ...,  0.0605, -0.0418, -0.0449],
        [ 0.0048,  0.0301,  0.0065,  ...,  0.0598, -0.0422, -0.0453],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
model.init_weights()

In [ ]:
model.state_dict()["final_logits_bias"]

tensor([[ 6.3965, -0.8680, -2.8505,  ..., -2.2678, -2.2489,  0.0000]])

In [ ]:
model.state_dict()[s]

tensor([[-0.0175,  0.0060, -0.0120,  ..., -0.0074, -0.0449, -0.0565],
        [ 0.0209,  0.0023, -0.0043,  ...,  0.0590, -0.0862, -0.0706],
        [-0.0041,  0.0053,  0.0189,  ...,  0.0026, -0.0172, -0.0435],
        ...,
        [ 0.0054,  0.0304,  0.0070,  ...,  0.0605, -0.0418, -0.0449],
        [ 0.0048,  0.0301,  0.0065,  ...,  0.0598, -0.0422, -0.0453],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
model.device

device(type='cpu')

In [ ]:
model.to(device)

In [ ]:
o_zh_text = "今天天气不错" # 原数据
print("o_zh: ", o_zh_text)
en_text = trans(model, tokenizer, o_zh_text)
print("en: ", en_text) ## 翻译为英文

o_zh:  今天天气不错


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


en:  It's a nice day.


In [ ]:
tokenizer.tokenize(o_zh_text)

['▁今天', '天气', '不错']

这个模型的效果太好了，我们不期望这么好的模型，放弃使用预训练模型，从AutoConfig开始初始化模型

## 数据集

汉译英数据集

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter7/language_tags.png)

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="zh_CN")
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
train_dataset = split_datasets["train"]
test_dataset = split_datasets["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 139666
    })
})

In [ ]:
raw_datasets['train'][0:3]

{'id': ['0', '1', '2'],
 'translation': [{'en': 'ROLES_OF_TRANSLATORS', 'zh_CN': 'Funda Wang'},
  {'en': 'CREDIT_FOR_TRANSLATORS', 'zh_CN': '开源软件国际化之简体中文组'},
  {'en': 'ROLES_OF_TRANSLATORS', 'zh_CN': 'Funda Wang'}]}

In [ ]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 125699
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 13967
    })
})

In [ ]:
# split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
train_dataset[0]

{'id': '114557', 'translation': {'en': 'Download', 'zh_CN': '下载'}}

In [ ]:
max_length = 128

def preprocess_function(examples):
  inputs = [ex["zh_CN"] for ex in examples["translation"]]
  targets = [ex["en"] for ex in examples["translation"]]
  return tokenizer(
      inputs, text_target=targets, max_length=max_length, truncation=True
  )

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

test_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names,
)

Map:   0%|          | 0/125699 [00:00<?, ? examples/s]

Map:   0%|          | 0/13967 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0], test_dataset[0]

({'input_ids': [7, 25618, 0],
  'attention_mask': [1, 1, 1],
  'labels': [53351, 0]},
 {'input_ids': [7, 9120, 0],
  'attention_mask': [1, 1, 1],
  'labels': [27021, 0]})

In [ ]:
from transformers import AutoModelForSeq2SeqLM
# model_checkpoint = f"Helsinki-NLP/opus-mt-zh-en"
model_checkpoint = "jieshenai/zh_en_translation"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([train_dataset[i] for i in range(0, 3)])
batch

{'input_ids': tensor([[    7, 25618,     0, 65000, 65000, 65000, 65000, 65000, 65000],
        [    7,   622,    59,     7,    11,  9587,   627, 56848,     0],
        [    7, 46277,     0, 65000, 65000, 65000, 65000, 65000, 65000]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[53351,     0,  -100,  -100,  -100,  -100,  -100,  -100],
        [54581,  2638, 44518,    14,     7,   622,    59,     0],
        [  456,     0,  -100,  -100,  -100,  -100,  -100,  -100]]), 'decoder_input_ids': tensor([[65000, 53351,     0, 65000, 65000, 65000, 65000, 65000],
        [65000, 54581,  2638, 44518,    14,     7,   622,    59],
        [65000,   456,     0, 65000, 65000, 65000, 65000, 65000]])}

## 训练

## 评价

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    # f"marian-finetuned-kde4-en-to-zh",
    "jieshenai/zh_en_translation",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

In [ ]:
train_dataset[0]

{'id': '114557', 'translation': {'en': 'Download', 'zh_CN': '下载'}}

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

Cloning https://huggingface.co/jieshenai/zh_en_translation into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/296M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/296M [00:00<?, ?B/s]

In [ ]:
trainer.evaluate(max_length=max_length)

***** Running Evaluation *****
  Num examples = 13967
  Batch size = 64


{'eval_loss': 1.580780029296875,
 'eval_bleu': 33.162868190029684,
 'eval_runtime': 1000.2055,
 'eval_samples_per_second': 13.964,
 'eval_steps_per_second': 0.219}

预训练模型结果

```python
{
 'eval_loss': 0.8558505773544312,
 'eval_bleu': 52.94161337775576,
 'eval_runtime': 714.2576,
 'eval_samples_per_second': 29.426,
 'eval_steps_per_second': 0.461,
 'epoch': 3.0
 }
```

随机初始化后的结果

```python
{
'eval_loss': 1.580780029296875,
 'eval_bleu': 33.162868190029684,
 'eval_runtime': 1000.2055,
 'eval_samples_per_second': 13.964,
 'eval_steps_per_second': 0.219
}
```

经过数据集训练之后

```python
***** Running Evaluation *****
  Num examples = 13967
  Batch size = 64

[219/219 20:19]

{'eval_loss': 1.142004132270813,
 'eval_bleu': 42.28398017121882,
 'eval_runtime': 1251.9837,
 'eval_samples_per_second': 11.156,
 'eval_steps_per_second': 0.175,
 'epoch': 3.0}
```

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 125699
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 11787
  Number of trainable parameters = 77419008


Step,Training Loss
500,0.930500
1000,0.973300
1500,0.982900
2000,0.977700
2500,0.989800
3000,0.983800
3500,0.992100
4000,0.981200
4500,0.858400
5000,0.876700


Saving model checkpoint to jieshenai/zh_en_translation/checkpoint-3929
Configuration saved in jieshenai/zh_en_translation/checkpoint-3929/config.json
Model weights saved in jieshenai/zh_en_translation/checkpoint-3929/pytorch_model.bin
tokenizer config file saved in jieshenai/zh_en_translation/checkpoint-3929/tokenizer_config.json
Special tokens file saved in jieshenai/zh_en_translation/checkpoint-3929/special_tokens_map.json
tokenizer config file saved in jieshenai/zh_en_translation/tokenizer_config.json
Special tokens file saved in jieshenai/zh_en_translation/special_tokens_map.json
Saving model checkpoint to jieshenai/zh_en_translation/checkpoint-7858
Configuration saved in jieshenai/zh_en_translation/checkpoint-7858/config.json
Model weights saved in jieshenai/zh_en_translation/checkpoint-7858/pytorch_model.bin
tokenizer config file saved in jieshenai/zh_en_translation/checkpoint-7858/tokenizer_config.json
Special tokens file saved in jieshenai/zh_en_translation/checkpoint-7858/spec

TrainOutput(global_step=11787, training_loss=0.90563847299163, metrics={'train_runtime': 3334.4011, 'train_samples_per_second': 113.093, 'train_steps_per_second': 3.535, 'total_flos': 5451854169047040.0, 'train_loss': 0.90563847299163, 'epoch': 3.0})

In [ ]:
trans(model, tokenizer, o_zh_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'It was a nice day.'

In [ ]:
trans(model, tokenizer, "午间新闻播报，今日在武汉市有歹徒上街杀人并强奸数位女性。该歹徒随后被警方击毙。")

'Newscast on noon today in Wuhan city when a mob killed a few of them and killed several of them. The mob was shot and killed by the police.'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.15.1
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: datasets, evaluate, transformers


In [ ]:
trainer.push_to_hub(
    repo_id="jieshenai/zh_en_translation",
    tags="translation",
    commit_message="trainer",
    # create_pr=True,
    )

TypeError: ignored

In [ ]:
# model.push_to_hub(tags="translation", commit_message="zh_en")

TypeError: ignored

In [ ]:
%cd marian-finetuned-kde4-en-to-zh

/content/marian-finetuned-kde4-en-to-zh


In [ ]:
!tar -zcvf checkpoint-11787.tar.gz checkpoint-11787

checkpoint-11787/
checkpoint-11787/pytorch_model.bin
checkpoint-11787/special_tokens_map.json
checkpoint-11787/rng_state.pth
checkpoint-11787/tokenizer_config.json
checkpoint-11787/vocab.json
checkpoint-11787/training_args.bin
checkpoint-11787/trainer_state.json
checkpoint-11787/config.json
checkpoint-11787/source.spm
checkpoint-11787/target.spm
checkpoint-11787/optimizer.pt
checkpoint-11787/scheduler.pt


In [ ]:
!cp checkpoint-11787.tar.gz /content/drive/MyDrive/tmp/

In [ ]:
trainer.evaluate(max_length=max_length)

***** Running Evaluation *****
  Num examples = 13967
  Batch size = 64


{'eval_loss': 1.142004132270813,
 'eval_bleu': 42.28398017121882,
 'eval_runtime': 1251.9837,
 'eval_samples_per_second': 11.156,
 'eval_steps_per_second': 0.175,
 'epoch': 3.0}

In [ ]:
model.push_to_hub(
    repo_id="jieshenai/zh_en_translation",
    create_pr=True,
    )

Configuration saved in /tmp/tmpn4m4bgnf/config.json
Model weights saved in /tmp/tmpn4m4bgnf/pytorch_model.bin
Uploading the following files to jieshenai/zh_en_translation: pytorch_model.bin,config.json


CommitInfo(commit_url='https://huggingface.co/jieshenai/zh_en_translation/commit/45cf2bcd71380166722c5f9731972e471d300127', commit_message='Upload model', commit_description='', oid='45cf2bcd71380166722c5f9731972e471d300127', pr_url='https://huggingface.co/jieshenai/zh_en_translation/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
tokenizer.push_to_hub(
    repo_id="jieshenai/zh_en_translation",
    create_pr=True,
    )

tokenizer config file saved in /tmp/tmpnh78gmtn/tokenizer_config.json
Special tokens file saved in /tmp/tmpnh78gmtn/special_tokens_map.json
Uploading the following files to jieshenai/zh_en_translation: special_tokens_map.json,tokenizer_config.json,vocab.json,source.spm,target.spm


CommitInfo(commit_url='https://huggingface.co/jieshenai/zh_en_translation/commit/0f430ccb8cbb591b036f7741a11901c43f9f763c', commit_message='Upload tokenizer', commit_description='', oid='0f430ccb8cbb591b036f7741a11901c43f9f763c', pr_url='https://huggingface.co/jieshenai/zh_en_translation/discussions/2', pr_revision='refs/pr/2', pr_num=2)